# Import Statements

In [1]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cv2 as cv
import scipy.io
from matplotlib import path
import pandas as pd

# import images
mask_dir = 'Implant Outlines/'
image_dir = 'Input Images/'
export_dir = 'Output Images/'

# Analysis Module

In [2]:
class Histology_CAT: 
    
    def __init__(self, image_dir, export_dir, mask_dir = None, debug = False):
        
        self.export_dir = export_dir #directory where processed images should be deposited
        self.mask_dir = mask_dir #directory where polygonal x-y mask points should be deposited
        self.debug = debug #determine whether or not to output all intermediate images
        self.image_dict = {}
        self.image_key = None
    
        #generate dictionary of images from the directory of images
        if self.mask_dir is None:
            for file in os.listdir(image_dir): 
                self.image_dict[os.path.basename(file)] = np.asarray(Image.open(image_dir + file))
        else: 
            mask_dict = {}
            for file in os.listdir(mask_dir):
                mask_id, _ = file.split('_') #Characters before '_' delimiter in a file name are the id
                matlab_dict = scipy.io.loadmat(mask_dir+file)
                mask_dict[mask_id] = np.asarray(matlab_dict[list(matlab_dict)[3]])
                
            for file in os.listdir(image_dir):
                image = np.asarray(Image.open(image_dir + file))
                image_id, _ = file.split('_')
                try: 
                    self.image_dict[os.path.basename(file)] = (image, mask_dict[image_id])
                except: 
                    print(f"Couldn't find polygonal mask points associated with {file}")
                    
            self.metrics_df = pd.DataFrame(index = self.image_dict.keys(), columns = ['tissue pixel count', 'implant pixel count', 'empty space pixel count', '% infill', '% porosity'])
            
    
    #process all images in the image directory with only n = 2 clusters
    def kowalski(self, krnl_sz):
        
        for image_key in self.image_dict:
            self.image_key = image_key
            
            #kmeans clustering to separate into tissue and non-tissue clusters
            if self.mask_dir is not None:
                _, polygonal_mask = self.polygon()
                kmeans_arr, centers = self.kmeans_quant(self.image_dict[image_key][0], n_cluster = 2, masks = [polygonal_mask])
            else:
                kmeans_arr, centers = self.kmeans_quant(self.image_dict[image_key], n_cluster = 2) 
            tissue_rgb = centers[np.argmin(np.sum(centers, axis = 1))] #identify RGB color of the tissue (the darker color)
            tissue_rgb = tissue_rgb.astype('uint8')
        
            #obtain tissue_mask
            tissue_mask = self.cat_mask(kmeans_arr, tissue_rgb)
            
            #kmeans clustering to separate empty space and implant clusters
            if self.mask_dir is not None:
                notissue_arr = self.img_overwrite(self.image_dict[image_key][0], tissue_mask, (255,255,255)) #overwrite tissue pixels with rgb (255, 255, 255) to match empty space
                kmeans_arr, centers = self.kmeans_quant(notissue_arr, n_cluster = 2, masks = [polygonal_mask])
            else:
                notissue_arr = self.img_overwrite(self.image_dict[image_key], tissue_mask, (255,255,255)) #overwrite tissue pixels with rgb (255, 255, 255) to match empty space
                kmeans_arr, centers = self.kmeans_quant(notissue_arr, n_cluster = 2) 
            implant_rgb = centers[np.argmin(np.sum(centers, axis = 1))] #identify RGB color of the implant (the darker color)
            implant_rgb = implant_rgb.astype('uint8')
            
            #obtain implant mask
            implant_mask = self.cat_mask(kmeans_arr, implant_rgb)
            
            #assemble recolored image by overwriting an all-white array with tissue and implant pixels
            if self.mask_dir is not None:
                white_img = np.ones(self.image_dict[image_key][0].shape)*255
                white_img = white_img.astype('uint8')
                white_img = self.img_overwrite(white_img, np.logical_not(polygonal_mask), (220,220,220)) #set background to grey
            else:
                white_img = np.ones(self.image_dict[image_key].shape)*255
                white_img = white_img.astype('uint8')
            tissue_overwrite = self.img_overwrite(white_img, tissue_mask, (120,81,169)) #set tissue to purple
            implant_overwrite = self.img_overwrite(tissue_overwrite, implant_mask, (0,0,0)) #set implant to black
            
            #apply median filter to the overwritten image
            filtered_img = self.med_filter(implant_overwrite, krnl_sz)
            
            #save final image
            final_img = Image.fromarray(filtered_img, mode = 'RGB') 
            final_img.save(self.export_dir + "Analyzed " + self.image_key, "JPEG")
            
            #store metrics of interest in dataframe
            if self.mask_dir is not None:
                num_p = np.sum(tissue_mask)
                num_b = np.sum(implant_mask)
                num_w = np.sum(polygonal_mask) - num_p - num_b
                infill = num_p/(num_p + num_w) * 100
                porosity = (num_p + num_w)/(num_p + num_w + num_b) * 100
                
                self.metrics_df.loc[self.image_key] = [num_p, num_b, num_w, infill, porosity]
                
        #export dataframe with metrics of interest
        if self.mask_dir is not None: 
            self.metrics_df.to_csv(self.export_dir + 'metrics.csv')
     
        return "All done, Skipper!"
       
    
    #perform K Means clustering to quantize RGB colors in the image 
    def kmeans_quant(self, img_array, n_cluster, masks = None):
        
        flat_array = img_array.reshape(-1,3)
        
        #if masking, identify indices where the mask is true in the original array
        if masks is not None: 
            orig_array = img_array.reshape(-1,3).copy() #needs to be a copy since we are overwriting
            total_mask = masks[0]
            for mask in masks: 
                total_mask = np.logical_and(total_mask, mask)
            indices = np.where(total_mask)
            flat_array = flat_array[total_mask]
        
        #K-Means
        clustering = KMeans(n_clusters=n_cluster, random_state=42, n_init = 10).fit(flat_array)
        labels = clustering.labels_
        centers = clustering.cluster_centers_ 
        
        if masks is not None: 
            #Placing clusters from masked image into original image
            orig_array[indices] = centers[labels] 
            quantized_array = orig_array.reshape(img_array.shape).astype('uint8')
        else: 
            quantized_array = centers[labels].reshape(img_array.shape).astype('uint8')
       
        #Save image after application of K Means clustering quantization
        if self.debug:
            quantized_img = Image.fromarray(quantized_array, mode = 'RGB') 
            quantized_img.save(self.export_dir + str(n_cluster) + " quanta " + self.image_key, "JPEG")
    
        return quantized_array, centers
    
    
    #perform median filtering to smooth local blemishes while keeping edges
    def med_filter(self, image_arr, krnl_sz):
        
        filtered_arr = cv.medianBlur(image_arr, krnl_sz)
        
        #Save image after application of median filtering
        if self.debug:
            filtered_img = Image.fromarray(filtered_arr, mode = 'RGB') 
            filtered_img.save(self.export_dir + str(krnl_sz) + " pixel median filter "+ self.image_key, "JPEG")
        
        return filtered_arr
    
    
    #given a pixel's category color (derived from K-means cluster center rgb), return boolean list of all corresponding pixels
    def cat_mask(self, image_arr, cat_color):
    
        bool_array = (image_arr == cat_color) #each data element is checked against cat_color
        bool_array = np.sum(bool_array, axis = 2) #sum boolean array of rgb values at each pixel
        bool_array = (bool_array == 3) # if r+g+b matches, the above sum is 3 and this pixel should be set to True
        
        return bool_array.ravel()
    
    
    def img_overwrite(self, image_arr, mask_array, rgb_color):
    
        overwrite_arr = image_arr.copy() #to prevent 'assignment destination is read-only' error
        overwrite_arr.reshape(-1,3)[mask_array] = [rgb_color]
        
        #Save image after application of naive categorical recoloring
        if self.debug:
            overwrite_img = Image.fromarray(overwrite_arr, mode = 'RGB') 
            overwrite_img.save(self.export_dir + str(rgb_color) + " overwrite " + self.image_key, "JPEG")
        
        return overwrite_arr
    
    
    def polygon(self):
        
        #get the points for polygon   *note: imported points have origin in bottom left hand corner
        pts = self.image_dict[self.image_key][1].reshape(-1,2)
        x_pts = pts[:,0]
        y_pts = pts[:,1]
        
        #flip y_pts so that the points for the polygon will have the origin in the top left hand corner
        image = self.image_dict[self.image_key][0]
        y_max = image.shape[0] #yes, PIL truly does store images as (y,x) because convention is for weenie hut juniors
        y_pts = np.abs(y_pts - y_max)
        
        #reassemble points and make polygon path
        pts = np.column_stack((x_pts, y_pts)).reshape(-1,2)
        polygon = path.Path(pts, closed = True)
        
        #create x-y coordinates for pixels where the top left pixel is (0,0)
        y_dim, x_dim, _ = np.shape(self.image_dict[self.image_key][0]) #again, really is stored as (y,x) 
        grid_x, grid_y = np.meshgrid(np.arange(x_dim), np.arange(y_dim)) 
        grid = np.stack((grid_x, grid_y), axis = 2).reshape(-1,2)
        
        #return true if points on the grid are inside the drawn polygon
        polygonal_mask = polygon.contains_points(grid)

        return polygon, polygonal_mask

# Kowalski, Analysis!

In [3]:
categorizer = Histology_CAT(image_dir, export_dir, mask_dir)
categorizer.kowalski(krnl_sz = 5)

'All done, Skipper!'